# Lecture 2: Python for Scientific Computing (1.5 Hours) #

### ABSTRACT ###

In this Lecture, we will do yet another kind of stuff. **todo**

---

**TODO**

- explain ``dtype``
- useful constructors like ``arange`` and ``linspace``.

## Manipulating Arrays w/ NumPy and SciPy (40 Minutes) ##

So far, we've seen that Python offers flexible structures such as ``list`` and ``dict`` for storing data, but this flexibility comes at a cost. Consider, for instance, representing a matrix as a ``list`` of ``list`` instances.

In [1]:
x = [
    [0, 1],
    [1, 0]
]

In [2]:
x

[[0, 1], [1, 0]]

This "works" in the sense that it stores the data we're interested in, but fails to enforce that its entries are all numeric, that each "row" is the same length, and doesn't give us a terribly useful way of performing mathematical operations.

In [3]:
# Almost certainly not what we expect "matrix" addition to look like!
x + x

[[0, 1], [1, 0], [0, 1], [1, 0]]

In [4]:
# "Ragged" structures are OK for lists, even though that doesn't
# make sense for matrices.
x = [
    [0, 1, 0],
    [1, 0]
]
x

[[0, 1, 0], [1, 0]]

In [6]:
# Heterogeneuous data types are also OK for lists,
# but not what we want here.
x = [
    ["0", True],
    [1, None]
]
x

[['0', True], [1, None]]

None of this indicates that ``list`` is bad, but rather that it's not the data structure we want to represent tensors of numbers, such as vectors or matrices. Thankfully, the NumPy library provides precisely such a tensor structure, called an *array*. This library is installed by default with Anaconda, so let's go on and import it now.

In [7]:
import numpy as np
x = np.array([
    [0, 1],
    [1, 0]
])

In [8]:
x

array([[0, 1],
       [1, 0]])

In [9]:
x + x

array([[0, 2],
       [2, 0]])

In [10]:
2 * x

array([[0, 2],
       [2, 0]])

By contrast to ``list`` instances, arrays provide much more structure that we can use to represent numerical operations. The most basic operation allowed by arrays is that of *indexing*, wherein we access one or more elements of an array. This is similar to indexing lists, but much more powerful thanks to the additional assumptions that we can make about arrays. In particular, we can now index along multiple *dimensions* at once, each of which is also called an *axis* or an *index*.

In [17]:
x = np.array([
    [1, 2, 3],
    [4, 5, 6]
])
print(x[0, 1])

2


Each of these index expressions can itself be a slice, as we saw earlier with lists. The special slice ``:`` selects everything along that axis.

In [18]:
print(x[:, 0])
print(x[1, :])

[1 4]
[4 5 6]


In the example above, we note that indexing by a scalar removes that axis from the returned array, reducing the number of dimensions. This is represented in the array's *shape*, which describes the length of that array along each axis (this generalizes MATLAB's ``size`` concept to tensors of rank up to 63). Each shape is a ``tuple`` of integers, or the empty tuple for a scalar.

In [23]:
print(x.shape)
print(x[:, 0].shape)
print(x[1, :].shape)
print(x[1, :2].shape)
print(x[0, 0].shape)

(2L, 3L)
(2L,)
(3L,)
(2L,)
()


Arrays are also iterable, such that they can be used in ``for`` loops, ``map``, etc. There are often more powerful (and much faster!) ways of acting on arrays, as we'll see in a few minutes, but it's occasionally useful to iterate. When used in this way, an array iterates over its leftmost axis, yielding slices of the array. For example, if we think of a 2-axis array as a matrix whose first axis is a "row" and whose second index is a "column," then iterating yields row vectors as 1-axis arrays:

In [24]:
for vec in x:
    print(vec)

[1 2 3]
[4 5 6]


To iterate over an axis other than the leftmost, use the ``transpose`` method:

In [26]:
for col in x.transpose():
    print(col)

[1 4]
[2 5]
[3 6]


The ``transpose`` method is also exposed via the shorthand ``.T``:

In [28]:
np.all(x.transpose() == x.T)

True

In this example, if we did not call ``np.all``, we would get a Boolean array describing elementwise equality. For instance:

In [30]:
a = np.array([[1, 2], [3, 4]])
b = np.array([[0, 2], [0, 0]])
print(a == b)

[[False  True]
 [False False]]


The ``np.all`` function then *reduces* an array by using a logical ``and``, returning ``True`` if and only if all elements of its argument are ``True``. We can also reduce only over a subset of axes, using the ``axis`` argument:

In [34]:
a = np.array([[1, 1], [0, 1]])
b = np.array([[1, 1], [-1, 1]])

print(np.all(a == b, axis=0))
print(np.all(a == b, axis=1))

[False  True]
[ True False]


This tells us that the first "rows" of ``a`` and ``b`` are equal, while the second ``columns`` of ``a`` and ``b`` are equal. Other useful reductions include ``np.max``, ``np.min``, ``np.median``, and ``np.mean``.

In [37]:
vec = np.array([-10, 0, 1])
for reduction_fn in (np.mean, np.median, np.min, np.max):
    print(reduction_fn, reduction_fn(vec))

(<function mean at 0x0000000004E94828>, -3.0)
(<function median at 0x00000000069A3F98>, 0.0)
(<function amin at 0x0000000004E94438>, -10)
(<function amax at 0x0000000004E943C8>, 1)


Of course, we will often want to actually perform arithmetic operations on arrays. In NumPy, such operations are represented by *universal functions*, or *ufuncs* for short. Each ufunc describes an element-wise operation which NumPy can then act on arrays.

In [39]:
a = np.array([[0, 1], [2, 3]])
b = np.array([[0, 10], [20, 30]])

In [40]:
a + b

array([[ 0, 11],
       [22, 33]])

In [41]:
a * b

array([[ 0, 10],
       [40, 90]])

In [44]:
b ** a

array([[    1,    10],
       [  400, 27000]])

In [45]:
-a

array([[ 0, -1],
       [-2, -3]])

A ufunc can be applied to two or more arrays if their shapes match exactly, or if one can be *broadcast* to each other. Before we explain what broadcasting is in detail, let's jump right into an example: adding a scalar to a 2-axis array:

In [46]:
a + 1

array([[1, 2],
       [3, 4]])

Here, we see that NumPy knew to "repeat" the scalar 1 such that it can then properly match the shape ``(2, 2)`` of ``a``. This holds even for higher-rank arrays:

In [48]:
a = np.arange(6).reshape((2, 3, 1))
a

array([[[0],
        [1],
        [2]],

       [[3],
        [4],
        [5]]])

In [50]:
a ** 2

array([[[ 0],
        [ 1],
        [ 4]],

       [[ 9],
        [16],
        [25]]])

Essentially, broadcasting consists of a formal list of rules for determining when and how to repeat lower-rank arrays to properly match shapes for elementwise computation. First, the arrays being operated on by a ufunc have different numbers of axes, singleton dimensions (axes of length 1) are added to the left of each array until they have the same length shapes.
For instance, if ``a`` is a ``(1, 2)``-shape array, and ``b`` is a ``(2,)``-shape array, then ``a`` and ``b`` are treated as having the same shape for the purpose of broadcasting.

Next, for each axis, the lengths along that axis must match for each array, or any non-matching arrays must have length 1. Thus, an array of shape ``(1, 2)`` and an array of shape ``(3, 1)`` can be broadcast with each other, but ``(2, 2)`` does not broadcast with ``(3, 1)``.

As a more applied example, we can use broadcasting together with ``np.newaxis`` to quickly define generalized outer products:

In [52]:
a = np.array([1, 2, 3])
b = np.array([4, 5])

print(a[:, np.newaxis] * b)
print(a[:, np.newaxis] + b)
print(a[:, np.newaxis] ** b)
print(a[:, np.newaxis] % b)

[[ 4  5]
 [ 8 10]
 [12 15]]
[[5 6]
 [6 7]
 [7 8]]
[[  1   1]
 [ 16  32]
 [ 81 243]]
[[1 1]
 [2 2]
 [3 3]]


This works because ``np.newaxis`` adds a new singleton axis at that location in the indexing operation. In the example above, ``a[:, np.newaxis]`` has shape ``(3, 1)``, which broadcasts with ``b`` (shape ``(2,)``) to return a shape ``(3, 2)`` array. Broadcasting thus gives us a very powerful tool for quickly performing index gymnastics, even on high-rank tensors.

**TODO**

- Tensor operations (dot, contract).
- linalg (eig, for example)

## Pandas (15 Minutes) ##

- Loading data into data frames (NumPy arrays, CSV, Excel)
- Groupby / pivot, etc.
- Reductions

## Plotting with Matplotlib and Seaborn (25 Minutes) ##

- Basics of plotting, Jupyter Notebook support
- Style (``ggplot``, etc.)
- Modifying plots with labels, ticks, titles, etc.
- Bar plots, histograms, scatter.
- Seaborn for time series and joint plots.

## Unitful Computing with Pint and python-quantities (10 Minutes) ##

- Show brief examples of using p-q to detect unit/dimension problems
- Allude to IK, Mabuchi group.